In [51]:
! rm craigslist.csv; scrapy crawl craigslist --nolog -o craigslist.csv -t csv

In [52]:
! rm kijiji.csv; scrapy crawl kijiji --nolog -o kijiji.csv -t csv

In [53]:
import pandas as pd

craigslist = pd.read_csv('craigslist.csv')
kijiji = pd.read_csv('kijiji.csv')

df = craigslist.append(kijiji, ignore_index=True)

In [54]:
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

In [55]:
import pickle

def load_obj(name):
    with open('data/obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
distance = load_obj('distance')

In [56]:
lng_num, lat_num = 100, 100
min_lng, min_lat = distance[0]['lng'], distance[0]['lat']
max_lng, max_lat = distance[len(distance)-1]['lng'], distance[len(distance)-1]['lat']
eps_lng, eps_lat = (max_lng-min_lng)/(lng_num-1), (max_lat-min_lat)/(lat_num-1)

def sum_distance(lat, lng):
    if lat < min_lat:
        lat = min_lat
    if lat > max_lat:
        lat = max_lat
    if lng < min_lng:
        lng = min_lng
    if lng > max_lng:
        lng = max_lng
    lng_ind = int((lng - min_lng)/eps_lng)
    lat_ind = int((lat - min_lat)/eps_lat)
    total_ind = lat_ind * lng_num + lng_ind
    return max(distance[total_ind]['sfu_dis'], distance[total_ind]['ubc_dis'])

In [58]:
df2 = df[df['lat'].notnull()]
df2 = df2[df2['long'].notnull()]
df2['dis'] = df2.apply(lambda x: sum_distance(x['lat'], x['long']), axis=1)
df2 = df2.sort_values('dis')
df2 = df2[df2['dis'] > 0]
df2 = df2[df2['furnished']==True]
df2 = df2[df2['price'] < 1800]
df2.style.format({'url': make_clickable})

,url,price,lat,long,beds,area,furnished,available,dis
847,https://vancouver.craigslist.org/van/apa/d/vancouver-1-br-fully-furnished-and/6900886692.html,1750,49.2625,-123.086,1,625,True,jun 15,42
3221,https://www.kijiji.ca/v-room-rental-roommate/vancouver/1-room-available/1442254094,925,49.2626,-123.088,2,nan,True,nan,42
2085,https://vancouver.craigslist.org/van/apa/d/vancouver-cute-1-bedroom-plus-parking/6917918979.html,1750,49.2652,-123.089,1,500,True,nan,44
4701,https://www.kijiji.ca/v-room-rental-roommate/vancouver/500-room-for-rent-fully-furnished-needs-female-mount-pleasan/1442335603,500,49.2636,-123.078,nan,nan,True,nan,44
3191,https://www.kijiji.ca/v-commercial-office-space/vancouver/today-is-a-good-day-to-get-your-new-office/1431227148,1198,49.2879,-123.113,nan,nan,True,nan,45
200,https://vancouver.craigslist.org/van/apa/d/vancouver-room-for-rent/6918098801.html,820,49.2643,-123.076,3,nan,True,jul 1,45
3101,https://www.kijiji.ca/v-room-rental-roommate/vancouver/private-room/1442425287,400,49.285,-123.119,nan,nan,True,nan,45
3577,https://www.kijiji.ca/v-room-rental-roommate/vancouver/private-roomamazing-location/1441067862,950,49.274,-123.101,nan,nan,True,nan,45
4211,https://www.kijiji.ca/v-apartments-condos/vancouver/looking-for-place-to-rent-july-aug-2019/1438959950,1000,49.2827,-123.121,1,nan,True,nan,47
4346,https://www.kijiji.ca/v-room-rental-roommate/vancouver/fast-sameday-fob-copying-delivered-to-you/1365812560,35,49.2808,-123.115,nan,nan,True,nan,47
